##  実験目標
- 一局を通して現れる異常な行動を画像ベースで取得する

## 実験手順
- 棋譜から評価値を読み込む -> ok
- 「指し手から次の指し手」までの間隔を画像ベースで把握する -> ok
- 初手から10手をニュートラルの姿勢とし、異常検知をして画像を収集する -> ok
    - 初手から10手の画像とpose抽出画像をそれぞれaddWeightedで重ねる -> ok
    - 2種類の画像を基準に異常な行動画像を抽出
        - 10手で作成した合成画像から外れ値となるしきい値を生成

## モジュールの読み込み


In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.family'] = 'IPAPGothic'
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import imagehash
from PIL import Image

import os
import sys
from glob import glob

# 特徴量抽出
from DeNAPose.pose_detector import PoseDetector, draw_person_pose

from multiprocessing import Pool
import multiprocessing as mp


Bad key "\nfont.family" on line 618 in
/home/nvidia/.pyenv/versions/3.5.2/lib/python3.5/site-packages/matplotlib/mpl-data/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


Final1は藤井先生が後手なのでフラグをFalseへ

In [2]:
pose_thresh = 5 # openposeによるスケルトン抽出時の信頼度スコアのしきい値
sampling_frame_rate = 6 # サンプリングレート
isFujii = False

In [3]:
# データフレームの作成と出力
# 棋譜データの読み込みとデータフレーム作成
# kifデータを合わせる
with open("../data/kif/AbemaTV_Tournament_Final1_analized.kif", encoding="utf-8") as f:
    kif = f.readlines()
shogi_df = pd.DataFrame([int(k.split(" ")[1]) for k in kif if "*##" in k], columns=["score"])

if not isFujii:# 後手番なら評価値を逆にする
    shogi_df["score"] = shogi_df["score"] * -1
shogi_df['score_label'] = shogi_df["score"].map(lambda x: x//300)
shogi_df["hand_number"] = shogi_df.index

In [4]:
# 評価値が変化するまでと定義したときのthreshold
# 0が連続しているところまでを抽出
score0_idx = shogi_df[shogi_df.score == 0].index.values
threshold = score0_idx[np.where(np.diff(score0_idx) <= 3)[0][-1] + 1]

In [20]:
def trimming(img):
    mask = img < 255
    x = np.where(np.sum(mask, axis=0) > 1)[0]
    y = np.where(np.sum(mask, axis=1) > 1)[0]
    
    x_min, x_max = x[0], x[-1]
    y_min, y_max = y[0], y[-1]
    return x_min, x_max, y_min, y_max

In [21]:
def img_preprocess(img):
    size = 32
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # openposeのミスをできるだけ前処理で落とす
#     try:
    x, w, y, h = trimming(img)
    margin = 16
    img_trim = img[y-margin:h+margin, x-margin:w+margin]
    height, width = img_trim.shape

    if (height < 150) or (width < 150):
        return np.zeros((size, size))

    # 膨張処理
    kernel = np.ones((6,6),np.uint8)
    img_trim = cv2.bitwise_not(img_trim) # 白(255)を膨張させるため反転
    img_dil = cv2.dilate(img_trim,kernel,iterations = 1)

    img_dil = cv2.resize(img_dil, (size, size))

    # 2値化
    _,img_bin = cv2.threshold(img_dil,0,1,cv2.THRESH_BINARY)

    return img_bin * 255.0
#     except:
#         return np.zeros((size, size))

In [26]:
def extract(start, end, isFujii, output_path, cap):
    for i in tqdm(range(start, end), leave=False):
        cap.set(cv2.CAP_PROP_POS_FRAMES, sampling_frame_rate*i)
        frame = cv2.cvtColor(cap.read()[1], cv2.COLOR_BGR2RGB)
        pose_rect = cv2.flip(frame[678:968, 45:565].copy(), 1) if isFujii else frame[220:510, 1340:1860].copy()
        pose_keypoints, scores = pose_detector(pose_rect)

        if len(np.where(scores > pose_thresh)[0]) == 0:
            # 真っ白な画像を作成
            img_skel = np.ones_like(pose_rect, dtype=np.uint8) * 128
            img_skel = img_preprocess(img_skel)
        else:
            img_skel = draw_person_pose(np.full_like(pose_rect, 255), pose_keypoints[np.where(scores > pose_thresh)])
            img_skel = img_preprocess(img_skel)
        
        
        if not os.path.isdir(output_path):
            os.makedirs(output_path, exist_ok=True)

        cv2.imwrite(os.path.join(output_path, "img_frame{:06d}.png".format(sampling_frame_rate*i)), img_skel)

In [27]:
data_root_path = "/media/nvidia/JetsonTX2SSD240/data/"

In [24]:
pose_detector = PoseDetector("posenet", "../utils/DeNAPose/models/coco_posenet.npz", device=0)

Loading the model...


In [10]:
!ls /media/nvidia/JetsonTX2SSD240/data/

AbemaTV_Tournament_final1_abnorm_supervised.mp4
AbemaTV_Tournament_final1_abnorm_supervised_v2.mp4
AbemaTV_Tournament_Final1.mp4
AbemaTV_Tournament_Final2.mp4
AbemaTV_Tournament_Final3.mp4
final1_labeling.mp4
fujii_vs_kondo_Ablock_1_1080p.mov
fujii_vs_kondo_Ablock_2_1_1080p.mov
fujii_vs_kondo_Ablock_2_2_1080p.mov
fujii_vs_takami_semi_1_1080p.mov
fujii_vs_takami_semi_2_1080p.mov
fujii_vs_takami_semi_3_1080p.mov
shogi_pose_dataset
sup_or_inf
sup_or_inf_20181102
sup_or_inf_20181113
sup_or_inf_20181114
sup_or_inf_20181115


In [11]:
data_path = sorted(glob("/media/nvidia/JetsonTX2SSD240/data/AbemaTV_Tournament_final1_abnorm_supervised.mp4"))

data_path += sorted(glob(data_root_path + "*.mov"))

In [12]:
data_path

['/media/nvidia/JetsonTX2SSD240/data/AbemaTV_Tournament_final1_abnorm_supervised.mp4']

In [13]:
#fujii_bw = [False, True, False, True, False, False, True, False, True]
fujii_bw = [False]

In [28]:
for vid, fuji in zip(data_path, fujii_bw):
    output_path = os.path.join(data_root_path, "shogi_pose_dataset/{}".format(vid.split("/")[-1].split(".")[0]))
    print(output_path)
    os.makedirs(output_path, exist_ok=True)
    cap = cv2.VideoCapture(vid)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))//sampling_frame_rate
    extract(0, total, fuji, output_path, cap)

/media/nvidia/JetsonTX2SSD240/data/shogi_pose_dataset/AbemaTV_Tournament_final1_abnorm_supervised


def wrapper(args):
    return extract(*args)

with tqdm(total=total) as pbar:
    with Pool(mp.cpu_count()) as p:
        for _ in tqdm(enumerate(p.imap_unordered(wrapper, splits))):
            pbar.update()